# Table of Contents
 <p><div class="lev1 toc-item"><a href="#PMT/ADC" data-toc-modified-id="PMT/ADC-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PMT/ADC</a></div><div class="lev2 toc-item"><a href="#Example-of-how-to-compress-bytes-(e.g.,-JSON)-to-bzip2" data-toc-modified-id="Example-of-how-to-compress-bytes-(e.g.,-JSON)-to-bzip2-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Example of how to compress bytes (e.g., JSON) to bzip2</a></div><div class="lev1 toc-item"><a href="#Pump" data-toc-modified-id="Pump-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pump</a></div>

# PMT/ADC

In [14]:
import gtk
import gobject
import threading
import datetime as dt

import matplotlib as mpl
import matplotlib.style
import numpy as np
import pandas as pd

from streaming_plot import StreamingPlot


def _generate_data(stop_event, data_ready, data):
    delta_t = dt.timedelta(seconds=.1)
    samples_per_plot = 5

    while True:
        time_0 = dt.datetime.now()
        values_i = np.random.rand(samples_per_plot)
        absolute_times_i = pd.Series([time_0 + i * delta_t
                                      for i in xrange(len(values_i))])
        data_i = pd.Series(values_i, index=absolute_times_i)
        data.append(data_i)
        data_ready.set()
        if stop_event.wait(samples_per_plot *
                           delta_t.total_seconds()):
            break
        
with mpl.style.context('seaborn',
                       {'image.cmap': 'gray',
                        'image.interpolation' : 'none'}):
    win = gtk.Window()
    win.set_default_size(800, 600)
    view = StreamingPlot(data_func=_generate_data)
    win.add(view.widget)
    win.connect('check-resize', lambda *args: view.on_resize())
    win.set_position(gtk.WIN_POS_MOUSE)
    win.show_all()
    view.fig.tight_layout()
    win.connect('destroy', gtk.main_quit)
    gobject.idle_add(view.start)
    
    def auto_close(*args):
        if not view.stop_event.is_set():
            # User did not explicitly pause the measurement.  Automatically
            # close the measurement and continue.
            win.destroy()
    gobject.timeout_add(5000, auto_close)
    
    measurement_complete = threading.Event()
    
    view.widget.connect('destroy', lambda *args: measurement_complete.set())

    gtk.gdk.threads_init()
    gtk.gdk.threads_enter()
    gtk.main()
    gtk.gdk.threads_leave()
    
    print measurement_complete.wait()

True


## Example of how to compress bytes (e.g., JSON) to bzip2

In [ ]:
from IPython.display import display
import bz2


data = pd.concat(view.data)
data_json = data.to_json()
data_json_bz2 = bz2.compress(data_json)
data_from_json = pd.read_json(bz2.decompress(data_json_bz2), typ='series')
len(data_json), len(data_json_bz2)

------------------------------------------------------------------------

# Pump #

In [13]:
import threading
import logging; logging.basicConfig(level=logging.DEBUG)
import gobject
import gtk

from pygtkhelpers.delegates import SlaveView

logger = logging.getLogger(__name__)


class PumpControl(SlaveView):
    def __init__(self, proxy):
        self.proxy = proxy
        super(PumpControl, self).__init__()
    
    def create_ui(self):
        def _pump_frequency(adj):
            try:
                proxy.pump_frequency_set(adj.value)
            except:
                logger.debug('[_pump_frequency] Error setting pump frequency'
                             ' to %s.', adj.value)
            
        # value, lower, upper, step_increment, page_increment, page_size
        # Note that the page_size value only makes a difference for
        # scrollbar widgets, and the highest value you'll get is actually
        # (upper - page_size).
        self.frequency_adj = gtk.Adjustment(1e3, 1, 1e4 + 10, 0.1, 1.0, 10)
        self.frequency_scale = gtk.HScale(self.frequency_adj)
        
        self.duration_adj = gtk.Adjustment(3, 0, 120 + 10, 0.1, 1.0, 10)
        self.duration_scale = gtk.HScale(self.duration_adj)
        
        for label_i, scale_i in ((gtk.Label("Frequency (Hz):"),
                                  self.frequency_scale),
                                 (gtk.Label("Duration (s):"),
                                  self.duration_scale)):
            scale_i.set_size_request(200, 50)
            scale_i.set_update_policy(gtk.UPDATE_CONTINUOUS)
            scale_i.set_digits(1)
            scale_i.set_value_pos(gtk.POS_TOP)
            scale_i.set_draw_value(True)
            box_i = gtk.HBox()
            box_i.pack_start(label_i, False, False, 0)
            box_i.pack_start(scale_i, True, True, 0)
            self.widget.pack_start(box_i, False, False, 0)

        self.widget.show_all()

        self.frequency_adj.connect('value_changed', _pump_frequency)

        self.pump_button = gtk.Button('Pump')
        self.pump_button.connect('clicked', lambda *args: self.start())

        self.pump_cancel = gtk.Button('Cancel')
        self.pump_cancel.props.sensitive = False

        self.button_box = gtk.HBox()
        self.widget.pack_start(self.button_box, True, True)
        for widget_i in (self.pump_button, self.pump_cancel):
            self.button_box.pack_start(widget_i, True, True)
        self._enable_widgets()

    def _enable_widgets(self):
        for widget_i in (self.frequency_scale, self.duration_scale,
                         self.pump_button):
            widget_i.props.sensitive = True
        self.pump_cancel.props.sensitive = False

    def _disable_widgets(self):
        for widget_i in [self.duration_scale, self.pump_button]:
            widget_i.props.sensitive = False

    def start(self):
        cancel = threading.Event()

        def _cancel_pump(*args):
            cancel.set()
        gobject.idle_add(self.pump_cancel.connect, 'clicked',
                         _cancel_pump)

        def _pump():
            gobject.idle_add(self._disable_widgets)
            if self.proxy is not None:
                self.proxy.pump_activate()
            self.pump_cancel.props.sensitive = True
            try:
                if cancel.wait(self.duration_adj.value):
                    logger.warning('Pump action was cancelled.')
                else:
                    logger.info('Pump action completed.')
            finally:
                if self.proxy is not None:
                    self.proxy.pump_deactivate()
                gobject.idle_add(self._enable_widgets)
        thread = threading.Thread(target=_pump)
        thread.daemon = True
        thread.start()


win = gtk.Window()
win.set_modal(True)
win.set_default_size(480, 150)

pump_control_view = PumpControl(None)
win.add(pump_control_view.widget)
win.set_position(gtk.WIN_POS_MOUSE)
win.show_all()
gobject.idle_add(pump_control_view.start)
win.connect('destroy', gtk.main_quit)
gtk.gdk.threads_init()
gtk.main()

INFO:__main__:Pump action completed.
